In [1]:
import os

COLAB = False
if 'google.colab' in str(get_ipython()):
    COLAB = True

if COLAB:
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    repo_path = '/content/drive/Othercomputers/My Mac/266-implicit-hate-speech-detection'

    hf_token = userdata.get('hf_token')

else:
    repo_path = '..'

!python -m pip install nlpaug transformers sacremoses

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 19.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import nlpaug.augmenter.word as naw

In [3]:
processed_dir = os.path.join(repo_path, 'data/processed')
augment_dir = os.path.join(repo_path, 'data/back_translation')
orig_train_file = os.path.join(processed_dir, '%s/%s_train.csv')
save_file = os.path.join(augment_dir, '%s/%s_train.csv')

datasets = ['ihc', 'ishate']

## back_translation


In [4]:
back_translate = naw.back_translation.BackTranslationAug(
    device='cuda',
    batch_size=40,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

...

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

In [5]:
for corpus in datasets:
    df = pd.read_csv(orig_train_file % (corpus, corpus))
    counts = df['label'].value_counts()
    counts = tuple(zip(counts.index, counts.values))

    print(counts)

    threshold = counts[1][1]
    l_label = counts[0][0]
    m_label = counts[1][0]
    s_label = counts[2][0]
    s_size = counts[2][1]

    df['orig_id'] = df['id']
    df['orig_cleaned_text'] = df['cleaned_text']


    l_sample = df.loc[df['label'] == l_label].sample(threshold, random_state = 42)
    l_sample['aug_method'] = 'sampled_from_largest_class'
    m_sample = df.loc[df['label'] == m_label].sample(frac=1)
    m_sample['aug_method'] = 'untouched'

    s_orig = df.loc[df['label'] == s_label].sample(frac=1)
    s_orig['aug_method'] = 'untouched'

    diff = threshold - s_size
    s_sample = s_orig.sample(diff, random_state=42, replace=True)
    s_sample['aug_method'] = 'back-translated'

    tqdm.pandas(desc=f'Applying augmentation for {corpus}_train.csv')
    s_sample['cleaned_text'] = s_sample['cleaned_text'].progress_apply(back_translate.augment)

    out = pd.concat([l_sample, m_sample, s_orig, s_sample])
    out = out.sample(frac=1, random_state=42).reset_index()
    out.to_csv(save_file % (corpus, corpus))
    print(out.shape)

((0, 9304), (2, 4970), (1, 762))


Applying augmentation for ihc_train.csv:   0%|          | 0/4208 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Applying augmentation for ihc_train.csv: 100%|██████████| 4208/4208 [1:36:56<00:00,  1.38s/it]


(14910, 9)
((0, 12508), (1, 7007), (2, 866))


Applying augmentation for ishate_train.csv:   0%|          | 0/6141 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Applying augmentation for ishate_train.csv: 100%|██████████| 6141/6141 [2:33:00<00:00,  1.49s/it]


(21021, 9)
